In [ ]:
# Import necessary libraries
from QuantConnect import *
from QuantConnect.DataSource import *
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Initialize QuantBook
qb = QuantBook()

# Define economic indicators (using FRED and SPY for stock market)
indicators = {
    "GDP_Growth": qb.AddData(Fred, "A191RL1Q225SBEA", Resolution.Daily).Symbol,  # Real GDP growth, quarterly
    "Unemployment_Rate": qb.AddData(Fred, "UNRATE", Resolution.Daily).Symbol,     # Unemployment rate, monthly
    "Inflation": qb.AddData(Fred, "CPIAUCNS", Resolution.Daily).Symbol,           # CPI, monthly
    "Interest_Rates": qb.AddData(Fred, "DGS10", Resolution.Daily).Symbol,         # 10-Year Treasury Yield, daily
    "Stock_Market": qb.AddEquity("SPY", Resolution.Daily).Symbol,                  # SPY returns as proxy
    "VIX": qb.AddData(Fred, "VIXCLS", Resolution.Daily).Symbol                    #VIX volatility index, daily
    # Note: Consumer Confidence not available in standard library; use CSV below
}


# Define time period
start_date = datetime(2016, 1, 1)
end_date = datetime.now()
# Fetch historical data for all indicators
symbols = list(indicators.values())
history = qb.History(symbols, start_date, end_date, Resolution.Daily)

# Process historical data into a combined DataFrame
combined_data = pd.DataFrame()
if not history.empty:
    for name, symbol in indicators.items():
        # Filter data for the current indicator
        if name == "Stock_Market":
            # For SPY, calculate daily returns
            symbol_data = history.loc[symbol][["close"]]
            symbol_data[name] = symbol_data["close"].pct_change()
        else:
            # For FRED data, use 'value' column
            symbol_data = history.loc[symbol][["value"]].rename(columns={"value": name})
        
        # Ensure datetime index is clean
        symbol_data.index = symbol_data.index.date
        # Add to combined DataFrame
        combined_data = combined_data.join(symbol_data[name], how="outer") if not combined_data.empty else symbol_data[[name]]
    
    # Forward fill missing values (e.g., for monthly/quarterly data)
    combined_data = combined_data.ffill()
    
    # Ensure index is datetime
    combined_data.index = pd.to_datetime(combined_data.index)
    
    # Set index name to "Date"
    combined_data.index.name = "Date"
    
    print("Combined DataFrame from QuantConnect:")
    display(combined_data.head())
else:
    print("No historical data retrieved from QuantConnect.")

# # Fallback: Load economic_indicators.csv if Consumer Confidence or other data is needed
# try:
#     csv_data = pd.read_csv("economic_indicators.csv", parse_dates=["Date"], index_col="Date")
#     print("CSV DataFrame:")
#     display(csv_data.head())
#     # Merge with QuantConnect data
#     combined_data = combined_data.join(csv_data, how="left").ffill()
#     print("Merged DataFrame (QuantConnect + CSV):")
#     display(combined_data.head())
# except FileNotFoundError:
#     print("economic_indicators.csv not found. Please upload the file or check the path.")




# Store the DataFrame in Object Store
if not combined_data.empty:
    # Convert DataFrame to CSV string
    csv_data = combined_data.to_csv()
    # Save to Object Store
    qb.ObjectStore.Save("Economic_Indicators", csv_data)
    print("Economic indicators data saved to Object Store under key 'Economic_Indicators'")



# Plot separate graphs for each indicator
if not combined_data.empty:
    for column in combined_data.columns:
        # Skip if column is entirely NaN
        if combined_data[column].isna().all():
            print(f"Skipping plot for {column}: No data available.")
            continue
        
        plt.figure(figsize=(10, 5))
        plt.plot(combined_data.index, combined_data[column], label=column)
        
        # Customize title and y-axis label based on indicator
        if column == "GDP_Growth":
            plt.title("Real GDP Growth Rate (2016-2025)")
            plt.ylabel("Annualized % Change")
        elif column == "Unemployment_Rate":
            plt.title("Unemployment Rate (2016-2025)")
            plt.ylabel("Percent")
        elif column == "Inflation":
            plt.title("Consumer Price Index (2016-2025)")
            plt.ylabel("CPI Index")
        elif column == "Interest_Rates":
            plt.title("10-Year Treasury Yield (2016-2025)")
            plt.ylabel("Yield (%)")
        elif column == "Stock_Market":
            plt.title("S&P 500 Daily Returns (2016-2025)")
            plt.ylabel("Daily Return")
        elif column == "VIX":
            plt.title("VIX Volatility Index (2016-2025)")
            plt.ylabel("Index")
        elif column == "Consumer_Confidence":
            plt.title("Consumer Confidence Index (2016-2025)")
            plt.ylabel("Index")
        else:
            plt.title(f"{column} (2016-2025)")
            plt.ylabel("Value")
        
        plt.xlabel("Date")
        plt.legend()
        plt.grid(True)
        plt.show()



# Save data to CSV for reference
if not combined_data.empty:
    combined_data.to_csv("economic_indicators_data.csv")
    print("Data saved to economic_indicators_data.csv")